<a href="https://colab.research.google.com/github/hojuly/study_RecSys/blob/master/example_rec_sys_movielens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 간단한 추천시스템 만들기: MovieLens Small Education Data

## Preprocess

In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import io
from google.colab import files

uploaded = files.upload()

Saving ratings.csv to ratings.csv
Saving movies.csv to movies.csv
Saving tags.csv to tags.csv


In [6]:
movies_df = pd.read_csv(io.StringIO(uploaded['movies.csv'].decode('utf-8')), index_col=0)
ratings_df = pd.read_csv(io.StringIO(uploaded['ratings.csv'].decode('utf-8')))
tags_df = pd.read_csv(io.StringIO(uploaded['tags.csv'].decode('utf-8')))

In [7]:
movies_df.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [8]:
print(ratings_df.shape)
ratings_df.head()

(100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
# pivot ratings into movie features
user_movie_matrix = ratings_df.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0)
 
# convert dataframe of movie features to scipy sparse matrix
sparse_mat = csr_matrix(user_movie_matrix.values)

In [12]:
user_movie_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,4.5,3.5,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,5.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.5,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,2.5,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
sum(list(user_movie_matrix[1].value_counts())[1:])

232

In [16]:
user_info_df = pd.DataFrame(
    data=[sum(list(user_movie_matrix[int(x)].value_counts())[1:]) for x in user_movie_matrix.columns],
          index = user_movie_matrix.columns, columns=['movies_rated']
)

In [17]:
user_info_df.head()

,movies_rated
userId,
1,232
2,29
3,39
4,216
5,44


In [18]:
movie_info_df = pd.DataFrame(
    data=[sum(list(user_movie_matrix.loc[int(x)].value_counts())[1:]) for x in user_movie_matrix.index],
          index = user_movie_matrix.index, columns=['users_rated']
)

In [19]:
movie_info_df.head()

,users_rated
movieId,
1,215
2,110
3,52
4,7
5,49


## Train Test Split

In [20]:
train_df, test_df = train_test_split(
    ratings_df,
    test_size=0.2,
    random_state=5
)

In [21]:
print(train_df.shape)
print(test_df.shape)

(80668, 4)
(20168, 4)


테스트셋에는 존재하지만 트레인셋에는 없는 영화 또는 사용자 수

In [22]:
# 집합 A - 집합 B => 집합 B에는 없고 집합 A에만 있는 item 

# userId
print("사용자: ", len(list(
    set(test_df['userId'].unique()) - set(train_df['userId'].unique()))
))

# movieId
print("영화: ", len(list(
    set(test_df['movieId'].unique()) - set(train_df['movieId'].unique()))
))

print("test set의 전체 영화 수: ",  len(test_df['movieId'].unique()))

사용자:  0
영화:  714
test set의 전체 영화 수:  5129


In [23]:
movies_not_included = list(
    set(test_df['movieId'].unique()) - set(train_df['movieId'].unique())
)
print(sorted(movies_not_included)[:10])

not_included_df = test_df[test_df.movieId.isin(movies_not_included)].sort_values(by='movieId')
print(not_included_df.head(10))

print("train set에 없고, test set에만 있는 영화 데이터 수: ", not_included_df.shape)

[220, 238, 263, 298, 310, 311, 324, 385, 391, 449]
       userId  movieId  rating   timestamp
97086     604      220     3.0   832080636
6349       43      238     5.0   848994876
4067       27      238     3.0   962685525
97412     606      263     4.0  1171328200
40597     275      298     2.0  1049076725
696         6      310     3.0   845556119
97105     604      311     3.0   832080690
13146      84      324     4.0   860397228
5070       33      385     4.0   939716559
6926       47      391     4.5  1496206564
train set에 없고, test set에만 있는 영화 데이터 수:  (762, 4)


## 간단한 추천알고리즘 만들기

### 1. 랜덤 예측

In [24]:
# 0.5 - 5.0사이의 숫자를 예측해야할 평점 수 만큼 generate
ratings_range = np.arange(0.5, 5.5, step=0.5)
ratings_range

array([0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ])

In [25]:
import random


pred_random = [random.choice(ratings_range) for x in range(len(test_df))]
pred_random[:10]

[0.5, 4.5, 4.0, 2.0, 3.5, 3.0, 2.5, 3.0, 3.5, 5.0]

In [28]:
test_df['pred_ratings_random'] = pred_random

In [29]:
test_df.head()

,userId,movieId,rating,timestamp,pred_ratings_random
92163,597,11,3.0,941558713,0.5
71427,459,72998,5.0,1520233439,4.5
993,7,8965,1.0,1106636095,4.0
6514,45,329,4.0,950740723,2.0
6011,42,1210,5.0,996258118,3.5


In [27]:
mse = mean_squared_error(
    y_true=test_df['rating'].values,
    y_pred=test_df['pred_ratings_random'].values
)
rmse = np.sqrt(mse)

print(mse, rmse)

3.772139032130107 1.9421995345818892


### 2. 영화 평균 평점기반 예측

In [30]:
train_movie_df = train_df.groupby('movieId').mean()

print(train_movie_df.shape)
train_movie_df.head()

(9010, 3)


,userId,rating,timestamp
movieId,,,
1,317.982353,3.920588,1.130840e+09
2,319.870588,3.423529,1.131061e+09
3,290.372093,3.348837,1.008155e+09
4,212.833333,2.583333,9.082910e+08
5,322.634146,2.975610,9.969173e+08


In [31]:
def avg_rating_prediction(training_set, x):
    """
    Train Set에 있으면 평균으로,
    Test Set에 없으면 1처럼 랜덤으로
    ---
    Return: 예측 평점
    """
    if x in training_set.index:
        pred_rating = training_set.loc[x]['rating']
    else:
        pred_rating = random.choice(ratings_range)
    return pred_rating

In [32]:
test_df['pred_rating_movie'] = test_df['movieId'].apply(
    lambda x: avg_rating_prediction(train_movie_df, x)
)

test_df.head()

,userId,movieId,rating,timestamp,pred_ratings_random,pred_rating_movie
92163,597,11,3.0,941558713,0.5,3.633333
71427,459,72998,5.0,1520233439,4.5,3.584416
993,7,8965,1.0,1106636095,4.0,3.187500
6514,45,329,4.0,950740723,2.0,3.413580
6011,42,1210,5.0,996258118,3.5,4.139241


In [33]:
mse = mean_squared_error(
    y_true=test_df['rating'].values,
    y_pred=test_df['pred_rating_movie'].values
)
rmse = np.sqrt(mse)

print(mse, rmse)

1.0328577236210916 1.0162960806876564


### 3. 사용자 평균 평점기반 예측

In [34]:
train_user_df = train_df.groupby('userId').mean()

print(train_user_df.shape)
train_user_df.head()

(610, 3)


,movieId,rating,timestamp
userId,,,
1,1854.230769,4.420513,9.649824e+08
2,68680.000000,3.923077,1.445715e+09
3,5494.205882,2.397059,1.306464e+09
4,1907.514620,3.578947,9.639730e+08
5,364.424242,3.696970,8.474351e+08


In [35]:
test_df['pred_rating_user'] = test_df['userId'].apply(
    lambda x: avg_rating_prediction(train_user_df, x)
)
test_df.head()

,userId,movieId,rating,timestamp,pred_ratings_random,pred_rating_movie,pred_rating_user
92163,597,11,3.0,941558713,0.5,3.633333,3.968571
71427,459,72998,5.0,1520233439,4.5,3.584416,4.300000
993,7,8965,1.0,1106636095,4.0,3.187500,3.218254
6514,45,329,4.0,950740723,2.0,3.413580,3.881029
6011,42,1210,5.0,996258118,3.5,4.139241,3.551136


In [36]:
mse = mean_squared_error(
    y_true=test_df['rating'].values,
    y_pred=test_df['pred_rating_user'].values
)
rmse = np.sqrt(mse)

print(mse, rmse)

0.8783376274362448 0.9371966855661862


### 4. 장르 기반 영화 평점 예측

In [37]:
# create user_movie matrix by only using train_df
train_user_movie_matrix = train_df.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0)

In [39]:
train_user_movie_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,4.5,3.5,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,2.5,4.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
genres_df = movies_df['genres'].str.get_dummies(sep='|')
print(genres_df.shape)
genres_df = genres_df.loc[train_df.movieId.unique()]
print(genres_df.shape)
genres_df.head()

(9742, 20)
(9010, 20)


,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
354,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
69844,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0
5151,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5325,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
65,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
# trainset에서 영화별 유저 평점 평균
train_movie_avg_ratings_df = train_user_movie_matrix.copy()
train_movie_avg_ratings_df = train_movie_avg_ratings_df.replace(0, np.NaN)
train_movie_avg_ratings_df = train_movie_avg_ratings_df.mean(axis = 1)

train_movie_avg_ratings_df.head()

movieId
1    3.920588
2    3.423529
3    3.348837
4    2.583333
5    2.975610
dtype: float64

In [42]:
# genres_df에서 해당 장르가 포함된 모든 영화 index를 가져와서, 해당 영화의 유저 평균 평점의 평균을 구해서 장르 평균 평점으로 활용
genres_avg_ratings_df = pd.DataFrame(index=genres_df.columns, columns=['avg_ratings'])

for genre in genres_avg_ratings_df.index:
    genre_avg_rating = train_movie_avg_ratings_df.loc[genres_df[genres_df[genre].isin([1])].index].mean()
    genres_avg_ratings_df.loc[genre]['avg_ratings'] = genre_avg_rating

genres_avg_ratings_df

,avg_ratings
(no genres listed),3.21474
Action,3.10272
Adventure,3.22065
Animation,3.47457
Children,3.09322
Comedy,3.17494
Crime,3.30098
Documentary,3.78313
Drama,3.42112
Fantasy,3.21012


In [43]:
def get_genre_avg_ratings(x):
    genres_list = movies_df.loc[x]['genres'].split('|')
    rating = 0
    for genre in genres_list:
        rating += genres_avg_ratings_df.loc[genre]['avg_ratings']
    
    return rating / len(genres_list)

In [44]:
tqdm.pandas()
test_df['pred_rating_genre'] = test_df['movieId'].progress_apply(
    lambda x: get_genre_avg_ratings(x)
)

100%|██████████| 20168/20168 [00:07<00:00, 2692.66it/s]


In [45]:
test_df.head()

,userId,movieId,rating,timestamp,pred_ratings_random,pred_rating_movie,pred_rating_user,pred_rating_genre
92163,597,11,3.0,941558713,0.5,3.633333,3.968571,3.318066
71427,459,72998,5.0,1520233439,4.5,3.584416,4.300000,3.188983
993,7,8965,1.0,1106636095,4.0,3.187500,3.218254,3.264425
6514,45,329,4.0,950740723,2.0,3.413580,3.881029,3.250255
6011,42,1210,5.0,996258118,3.5,4.139241,3.551136,3.144122


In [46]:
mse = mean_squared_error(
    y_true=test_df['rating'].values,
    y_pred=test_df['pred_rating_genre'].values
)
rmse = np.sqrt(mse)

print(mse, rmse)

1.1165208586709656 1.0566555061470913


### 5. Rule 기반 영화 평점 예측

In [48]:
train_user_info_df = pd.DataFrame({
    'avg_ratings': train_df.groupby('userId')['rating'].mean(),
    'std_ratings': train_df.groupby('userId')['rating'].std(),
    'count_ratings': train_df.groupby('userId')['rating'].count()
})

train_user_info_df.head()

,avg_ratings,std_ratings,count_ratings
userId,,,
1,4.420513,0.771656,195
2,3.923077,0.820881,26
3,2.397059,2.113375,34
4,3.578947,1.318832,171
5,3.696970,1.015038,33


In [49]:
min_count = train_user_info_df['count_ratings'].min()
max_count = train_user_info_df['count_ratings'].max()
avg_count = train_user_info_df['count_ratings'].mean()

train_user_info_df['weights'] = train_user_info_df['count_ratings'].apply(
    lambda x: (x-avg_count) / (max_count-min_count)
)

In [50]:
train_user_info_df.head()

,avg_ratings,std_ratings,count_ratings,weights
userId,,,,
1,4.420513,0.771656,195,0.029135
2,3.923077,0.820881,26,-0.049323
3,2.397059,2.113375,34,-0.045609
4,3.578947,1.318832,171,0.017993
5,3.696970,1.015038,33,-0.046074


In [55]:
# 평점에선 normalize가 꼭 필요하진 않지만(범위가 한정적이기 때문)
# 다른 문제를 다룰 때엔 유용할 수 있음
from sklearn import preprocessing


min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(train_user_info_df)
df_normalized = pd.DataFrame(
    np_scaled,
    columns=train_user_info_df.columns,
    index=train_user_info_df.index
)

df_normalized.head()

,avg_ratings,std_ratings,count_ratings,weights
userId,,,,
1,0.847080,0.365130,0.085422,0.085422
2,0.715812,0.388422,0.006964,0.006964
3,0.313113,1.000000,0.010678,0.010678
4,0.625000,0.624041,0.074280,0.074280
5,0.656145,0.480293,0.010214,0.010214


In [57]:
df_normalized['normalized_avg_ratings'] = df_normalized['avg_ratings'] * 5
df_normalized.head()

,avg_ratings,std_ratings,count_ratings,weights,normalized_avg_ratings
userId,,,,,
1,0.847080,0.365130,0.085422,0.085422,4.235399
2,0.715812,0.388422,0.006964,0.006964,3.579060
3,0.313113,1.000000,0.010678,0.010678,1.565564
4,0.625000,0.624041,0.074280,0.074280,3.125000
5,0.656145,0.480293,0.010214,0.010214,3.280724


In [58]:
test_df['pred_rating_normalized'] = test_df['userId'].apply(
    lambda x: df_normalized.loc[x]['normalized_avg_ratings']
)

test_df.head()

,userId,movieId,rating,timestamp,pred_ratings_random,pred_rating_movie,pred_rating_user,pred_rating_genre,pred_rating_normalized
92163,597,11,3.0,941558713,0.5,3.633333,3.968571,3.318066,3.639087
71427,459,72998,5.0,1520233439,4.5,3.584416,4.300000,3.188983,4.076389
993,7,8965,1.0,1106636095,4.0,3.187500,3.218254,3.264425,2.649085
6514,45,329,4.0,950740723,2.0,3.413580,3.881029,3.250255,3.523580
6011,42,1210,5.0,996258118,3.5,4.139241,3.551136,3.144122,3.088305


In [59]:
mse = mean_squared_error(
    y_true=test_df['rating'].values,
    y_pred=test_df['pred_rating_normalized'].values
)
rmse = np.sqrt(mse)

print(mse, rmse)

1.132084261945097 1.063994483982458
